In [ ]:
from diffusers import StableDiffusionXLInpaintPipeline, AutoencoderKL
import torch
from PIL import Image, ImageOps
import numpy as np
import os

In [ ]:
vae = AutoencoderKL.from_pretrained("stabilityai/sdxl-vae", torch_dtype=torch.float16)
inpaint_pipe = StableDiffusionXLInpaintPipeline.from_single_file(
    "/home/oleksandr/projects/upwork/schouwey-shopify/models/juggerxlInpaint_juggerInpaintV8.safetensors", 
    original_config_file="/home/oleksandr/projects/upwork/schouwey-shopify/models/sd_xl_inpaint_config.yaml",
    vae=vae,
    torch_dtype=torch.float16).to("cuda")

In [ ]:
qrimg = Image.open("/home/oleksandr/projects/upwork/esov-api/tmp/qr/Piece 01.png")
style_fold = "/home/oleksandr/projects/upwork/esov-api/tmp/qr/Reference - General Style"
img_fold = "/home/oleksandr/projects/upwork/esov-api/tmp/qr/Style Scene 001"

style_imgs = []
for file in os.listdir(style_fold):
    style_img = Image.open(os.path.join(style_fold, file)).convert("RGB")
    style_imgs.append(style_img)

imgs = []
for file in os.listdir(img_fold):
    img = Image.open(os.path.join(img_fold, file)).convert("RGB")
    imgs.append(img)

print(f"QR Image size: {qrimg.size}")
print(f"Style Image size: {style_imgs[0].size}")
print(f"Image size: {imgs[0].size}")

In [ ]:
ImageOps.invert(qrimg.convert("L")).convert("L")

In [ ]:
# overlay img[0] and qrimg with qrimg mask and 0.5 blend
qr_img_resized = qrimg.resize(imgs[0].size).convert("RGBA")
to_blend = imgs[0].copy().convert("RGBA")
blended = Image.blend(to_blend, qr_img_resized, 0.7)
pasted = imgs[0].copy()
pasted.paste(blended, (0, 0), ImageOps.invert(qrimg.convert("L")).resize(imgs[0].size))
pasted

In [ ]:
w=1024
h=1024

generator = torch.Generator(device="cuda").manual_seed(1)
inpaint_pipe(
    "black color",
    negative_prompt="white color",
    image=pasted.resize((w, h)),
    mask_image=qrimg.convert("L"),
    strength=0.5,
    guidance_scale=20,
    num_inference_steps=20,
    generator=generator,
    w=w,
    h=h,
).images[0]